qgrid - An interactive grid for viewing and editing pandas DataFrames
=======================================================
Qgrid is an Jupyter notebook widget which uses a javascript library called SlickGrid to render pandas DataFrames within a Jupyter notebook. It was developed for use in [Quantopian's hosted research environment]( https://www.quantopian.com/research?utm_source=github&utm_medium=web&utm_campaign=qgrid-nbviewer).

The purpose of this notebook is to give an overview of what qgrid is capable of. Execute the cells below to generate some qgrids using a diverse set of DataFrames.

## Overview
* [SlickGrid](https://github.com/mleibman/SlickGrid) is a javascript grid which allows users to scroll, sort, 
and filter hundreds of thousands of rows with extreme responsiveness.  
* [Pandas](https://github.com/pydata/pandas) is a powerful data analysis / manipulation library for Python, and DataFrames are the primary way of storing and manipulating two-dimensional data in pandas.

[Qgrid](https://github.com/quantopian/qgrid) renders pandas DataFrames as SlickGrids, which enables users to explore the entire contents of a DataFrame using intuitive sorting and filtering controls.  It's built on the ipywidget framework and is designed to be used in Jupyter notebook, Jupyterhub, or Jupyterlab

## What's new in 1.0.3
* Added the ability to listen for events on all QgridWidget instances (using `qgrid.on`) as well as on individual instances (using `QgridWidget.on`).
* **Breaking API Change:** Previously the recommended (but not officially documented) way of attaching event handlers to a QgridWidget instance was to listen for changes to the ``_df`` attribute using the ``observe`` method (i.e.``qgrid_widget.observe(handle_df_changed, names=['_df'])``). This method will no longer work for most events (scrolling, sorting, filtering, etc) so the new ``QgridWidget.on`` method should be used instead.

## API & Usage
API documentation is hosted on [readthedocs](http://qgrid.readthedocs.io/en/widget-guidelines/). 

The API documentation can also be accessed via the "?" operator in IPython.  To use the "?" operator, type the name of the function followed by "?" to see the documentation for that function, like this:
```
qgrid.show_grid?
qgrid.set_defaults?
qgrid.set_grid_options?
qgrid.enable?
qgrid.disable?

```

## Example 1 - Render a DataFrame with many different types of columns

In [1]:
import numpy as np
import pandas as pd
import qgrid
randn = np.random.randn
df_types = pd.DataFrame({
    'A' : pd.Series(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06', '2013-01-07', '2013-01-08', '2013-01-09'],index=list(range(9)),dtype='datetime64[ns]'),
    'B' : pd.Series(randn(9),index=list(range(9)),dtype='float32'),
    'C' : pd.Categorical(["washington", "adams", "washington", "madison", "lincoln","jefferson", "hamilton", "roosevelt", "kennedy"]),
    'D' : ["foo", "bar", "buzz", "bippity","boppity", "foo", "foo", "bar", "zoo"] })
df_types['E'] = df_types['D'] == 'foo'
qgrid_widget = qgrid.QgridWidget(df=df_types, show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 150, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, id='a4d5cf2a-ef1e-4143-bc9a-5ccd10a9a3c9', precision=5, show_toolbar=True)

If you make any sorting/filtering changes, or edit the grid by double clicking, you can retrieve a copy of your DataFrame which reflects these changes by calling `get_changed_df` on the `QgridWidget` instance returned by `show_grid`.

In [2]:
qgrid_widget.get_changed_df()

,A,B,C,D,E
0,2013-01-01,0.632008,washington,foo,True
1,2013-01-02,-1.159340,adams,bar,False
2,2013-01-03,-0.941001,washington,buzz,False
3,2013-01-04,-0.395081,madison,bippity,False
4,2013-01-05,0.406320,lincoln,boppity,False
5,2013-01-06,-1.949846,jefferson,foo,True
6,2013-01-07,-0.254368,hamilton,foo,True
7,2013-01-08,0.492426,roosevelt,bar,False
8,2013-01-09,-0.922692,kennedy,zoo,False


## Example 2 - Render a DataFrame with 1 million rows
*Note: The reason for the redundant "import" statements in the next cell (and many subsequent cells) is because it allows us to run the cells in any order.*

In [3]:
import pandas as pd
import numpy as np
import qgrid

# set the default max number of rows to 10 so the larger DataFrame we render don't take up to much space 
qgrid.set_grid_option('maxVisibleRows', 10)

df_scale = pd.DataFrame(np.random.randn(1000000, 4), columns=list('ABCD'))
# duplicate column B as a string column, to test scalability for text column filters
df_scale['B (as str)'] = df_scale['B'].map(lambda x: str(x))
q_scale = qgrid.show_grid(df_scale, show_toolbar=True, grid_options={'forceFitColumns': False, 'defaultColumnWidth': 200})
q_scale

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defaultColumnWidth': 200, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 10, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, id='3f6a819f-3da7-4851-a77e-0da886da1a58', precision=5, show_toolbar=True)

In [4]:
q_scale.get_changed_df()

,A,B,C,D,B (as str)
0,-0.022598,0.646851,-0.315470,0.756979,0.6468511683068416
1,1.578137,-0.796227,0.381622,-0.781703,-0.7962270189850903
2,1.085122,1.364377,-0.328479,-0.202609,1.3643766698472795
3,-0.315499,0.904426,-0.745649,0.122556,0.9044260054769852
4,0.489007,1.480414,-0.399344,0.791736,1.4804139829592091
5,-1.804991,1.405854,0.327702,-0.829124,1.4058541593216156
6,0.719390,-1.819837,0.473553,1.125923,-1.8198373708458047
7,-2.964972,0.818498,-1.589457,-0.196770,0.8184983803835738
8,-0.882162,-0.699471,-2.090860,0.755076,-0.6994713739472752
9,-0.531267,-0.369908,0.016454,0.430180,-0.36990767056999657


## Example 3 - Render a DataFrame returned by Yahoo Finance by enabling automatic qgrids

In [5]:
import pandas as pd
import numpy as np
import qgrid
randn = np.random.randn

# Get a pandas DataFrame containing the daily prices for the S&P 500 from 1/1/2014 - 1/1/2017
from pandas_datareader.data import DataReader
spy = DataReader(
    'SPY',
    'yahoo',
    pd.Timestamp('2014-01-01'),  
    pd.Timestamp('2017-01-01'),
)
# Tell qgrid to automatically render all DataFrames and Series as qgrids.
qgrid.enable()

# Render the DataFrame as a qgrid automatically
spy

ModuleNotFoundError: No module named 'pandas_datareader'

In [ ]:
# Disable automatic display so we can display DataFrames in the normal way
qgrid.disable()

## Example 4 - Render a DataFrame with a multi-index

Create a sample DataFrame using the `wb.download` function and render it without using qgrid

In [ ]:
import qgrid
import pandas as pd
from pandas_datareader import wb
df_countries = wb.download(indicator='NY.GDP.PCAP.KD', country=['all'], start=2005, end=2008)
df_countries.columns = ['GDP per capita (constant 2005 US$)']
qgrid.show_grid(df_countries)

In [ ]:
df_countries

## Example 5 - Render a DataFrame with an interval column
Create a sample DataFrame using the `wb.download` function and render it without using qgrid

In [ ]:
import numpy as np
import pandas as pd
import qgrid

td = np.cumsum(np.random.randint(1, 15*60, 1000))
start = pd.Timestamp('2017-04-17')
df_interval = pd.DataFrame(
    [(start + pd.Timedelta(seconds=d)) for d in td],
    columns=['time'])

freq = '15Min'
start = df_interval['time'].min().floor(freq)
end = df_interval['time'].max().ceil(freq)
bins = pd.date_range(start, end, freq=freq)

df_interval['time_bin'] = pd.cut(df_interval['time'], bins)

qgrid.QgridWidget(df=df_interval, show_toolbar=True)

In [ ]:
df_interval

## Example 6 - Render a DataFrame with unnamed columns
Create a sample DataFrame using the `wb.download` function and render it without using qgrid

In [ ]:
import numpy as np
import pandas as pd
import qgrid

arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
df_multi = pd.DataFrame(np.random.randn(8, 4), index=arrays)
qgrid.show_grid(df_multi, show_toolbar=True)

In [ ]:
df_multi

## Example 7 - Render a narrow DataFrame inside a Layout widget
Create a sample DataFrame with only two columns using `randint`, and render it in a Layout widget that's 20% of the width of the output area.

In [ ]:
import numpy as np
import pandas as pd
import qgrid
import ipywidgets as ipyw
randn = np.random.randn
df_types = pd.DataFrame(np.random.randint(1,14,14))
qgrid_widget = qgrid.QgridWidget(df=df_types, show_toolbar=False)
qgrid_widget.layout = ipyw.Layout(width='20%')
qgrid_widget

## Example 8 - Render a DataFrame with an index and column that contain multiple types

In [ ]:
import pandas as pd
import qgrid
df = pd.DataFrame({'A': [1.2, 'xy', 4], 'B': [3, 4, 5]})
df = df.set_index(pd.Index(['yz', 7, 3.2]))
view = qgrid.QgridWidget(df=df)
view

## Example 9 - Render a DataFrame with a Period index and Period column

In [ ]:
import pandas as pd
import qgrid
range_index = pd.period_range(start='2000', periods=10, freq='B')
df = pd.DataFrame({'a': 5, 'b': range_index}, index=range_index)
view = qgrid.QgridWidget(df=df)
view

## Example 10 - Render a DataFrame with NaN and None

In [ ]:
import pandas as pd
import numpy as np
import qgrid
df = pd.DataFrame([(pd.Timestamp('2017-02-02'), None, 3.4), (np.nan, 2, 4.7), (pd.Timestamp('2017-02-03'), 3, None)])
qgrid.show_grid(df)